# Conan Shing

## Research Interests:

My background is in integrative physiology research where I have been exposed to both the process of data collection and intial exploratory data analysis where descriptive stats, summary tables, and intial data visuilizations are produced. That intial EDA often involves heavy cleaning of our raw data before being imported in R studio for further analysis. My reserch intrests in my academic life revolve around identify sex differences in physiological responses in male and females, so no better comparrison than identifiying how the ratio of male to female medal holders compare across countries. My personal background is high performance sport, a passio I bring into the lab and my everyday life. As such there is no better data set than olympic results. My research questions are listed and summarized below: 

##  Q1: How does the ratio of male to female medal holders vary across countries. 

To answer this question I aim to compare the ammount of male vs female medla holders in various contries and identify if certain countries house predomenatly female or male medal winners. I first plan to use the medal results data, group by country and identify the number male and female medal holders for each country. From there I hope to identify which country containes the most male and most female medal holders. I would like to present this as a % of men and % of female medal holders so I will need to add a column for each country with the % of male and % of female medal holders. I would also like to see how this % changes over time in each country. I would like to present this as a horizontal bar graph showing cummilitive data and then a seperate line graph comparing the top 25 medal winning countires (over the entire time span of the data set) to compare how their ratio of male:female medal holders change.

## Q2: How does the country with the most medal holders at each game change over time and is this title associated with being the host country. 


To answer this question I plan to summarise the total number of medals attained at each game by country and identify the overall winner. From there I hope to compile a new sumarised data set where each country that attended the games have a total number of medals won where I then hope to plot all attendeing countries over time to see how the medal holding leaders change over time. I would also like to see if this has to do with being the host country that year. To do so I plan to compare avg medals won by non host countries attending the games and avg medals won by host countries. Out of intrest I would also like to see if this changes over time. 

For both my research questions some data wrangling will be needed. I most likely will combine the medal and host data sets to answer my second question and utilize the medals data set for my first. 



In [11]:
# setting up packages

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
# Set up visualization 

sns.set_theme(style="whitegrid")

In [13]:
# loading data sets

df_athlete = pd.read_csv("../../data/raw/Olympic_Summer_Winter_Games_1986-2022/olympic_athletes.csv")
df_hosts = pd.read_csv("../../data/raw/Olympic_Summer_Winter_Games_1986-2022/olympic_hosts.csv")
df_medals = pd.read_csv("../../data/raw/Olympic_Summer_Winter_Games_1986-2022/olympic_medals.csv")
df_results = pd.read_csv("../../data/raw/Olympic_Summer_Winter_Games_1986-2022/olympic_results.csv")

#df_athlete.head()
#df_hosts.head()
#df_medals.head()
#df_results.head()

In [46]:
# Q1 EDA

### creating working df
df = df_medals

### cleaning and wrangling data to prepare to add calculated columns

df['slug_game'] = df['slug_game'].str[-4:]
df = df.rename(columns={'slug_game': 'year'})
df = df.drop(['event_title', 'participant_title', 'athlete_url', 'athlete_full_name', 'country_code', 'country_3_letter_code'], axis = 'columns')

### calculating and creating medal stats 
df['medal_bysex'] = (df.groupby(['medal_type', 'event_gender', 'country_name', 'year'], as_index=False)['medal_type'].transform('count'))
df['medal_total'] = (df.groupby(['medal_type', 'country_name', 'year'], as_index=False)['medal_type'].transform('count'))
df['medal_all'] = (df.groupby(['country_name', 'year'], as_index=False)['medal_type'].transform('count'))
df['medal_%'] = ((df['medal_bysex']/df['medal_total'])*100)
df.sample(10)

### renaming df for future use
medal_bysex = df
medal_bysex
### Sanity check to make sure things are working right
medal_bysex.sample(10)



,discipline_title,year,event_gender,medal_type,participant_type,country_name,medal_bysex,medal_total,medal_all,medal_%
0,Taekwondo,2012,Men,BRONZE,Athlete,Afghanistan,1,1,1,100.0
1,Taekwondo,2008,Men,BRONZE,Athlete,Afghanistan,1,1,1,100.0
2,Sailing,1988,Open,SILVER,Athlete,Netherlands Antilles,1,1,1,100.0
3,Athletics,2016,Men,SILVER,Athlete,Algeria,2,2,2,100.0
4,Athletics,2016,Men,SILVER,Athlete,Algeria,2,2,2,100.0
...,...,...,...,...,...,...,...,...,...,...
21692,Swimming,2008,Women,GOLD,Athlete,Zimbabwe,1,1,4,100.0
21693,Swimming,2004,Women,GOLD,Athlete,Zimbabwe,1,1,3,100.0
21694,Swimming,2004,Women,BRONZE,Athlete,Zimbabwe,1,1,3,100.0
21695,Swimming,2004,Women,SILVER,Athlete,Zimbabwe,1,1,3,100.0


In [ ]:
# Q1 Data vis

# Who are the top 10 ever medal winners 

df = medal_bysex

df = df.sort_values(by=['medal_all'], ascending = False).groupby('year').head(10).reset_index()
df = df.drop(['index'], axis= "columns")
df = df.groupby('country_name', group_keys=False).apply(lambda x: x.loc[x.medal_all.idxmax()]).sort_values('medal_all',ascending = False).groupby('year').head(10)


plot = sns.barplot(x="medal_all", y="country_name", data=df,
                   label="Total Medals Won", color="b")
plot1.set(ylabel ="Number of Medals", xlabel ="Country Name", title = "Who are the top 10 ever medal winners")
plot

# next steps would be to see what portion of these resutls were from males vs females 
# I would like to create a facet grid plot where top 10 medal winners from each games were ploted every year with the
# males vs female contribution was provided

In [ ]:
#### Confirming all the unique years and dates are right. Intrestingly I found several gaps in the years of the games,
#### some of the presumably because of world wars or potentially due to errors in data collection 

#x = df_medals.slug_game.unique().tolist()
#x.sort()
#x

In [ ]:
# Q2 EDA

### To determine how a host country performs I will need to merge my data frames together

df1 = df_hosts

### preparing df_hosts for merge 
df1 = df1.drop(['game_slug', 'game_end_date', 'game_start_date', 'game_name'], axis = 'columns').rename(columns={'game_year' : 'year'})
df1.dtypes



In [ ]:
### preparing df_medals for merge

#the prep and primary wrangling was done above. 
df2 = medal_bysex
df2['year'] = df2['year'].astype(int)
#df2.dtypes

In [49]:
### merging both dataframes together

df_merge = df1.merge(df2, how='inner', on='year')
df_merge

#df_merge.sample(10)

### Comments: seems there are some countires that have held several olympics namely the US...this may skew the final
###           results and may be worth looking at it on a game by game basis. This would mean removing the over time clause
###           in my second research question. To be honest I don't think looking at time is all that useful. For example 
###           this would look like 

,game_location,game_season,year,discipline_title,event_gender,medal_type,participant_type,country_name,medal_bysex,medal_total,medal_all,medal_%
30302,Belgium,Summer,1920,Shooting,Men,BRONZE,GameTeam,Finland,9,9,35,100.000000
2148,Brazil,Summer,2016,Badminton,Women,SILVER,GameTeam,Denmark,4,7,18,57.142857
25992,Great Britain,Summer,1948,Cycling Track,Men,BRONZE,Athlete,Great Britain,6,8,29,75.000000
8630,United States,Summer,1996,Boxing,Men,GOLD,Athlete,Cuba,7,9,25,77.777778
2290,Brazil,Summer,2016,Taekwondo,Men,SILVER,Athlete,Great Britain,15,26,75,57.692308
10793,France,Winter,1992,Boxing,Men,SILVER,Athlete,Cuba,5,6,31,83.333333
9649,Spain,Summer,1992,Diving,Men,SILVER,Athlete,People's Republic of China,7,27,64,25.925926
31677,France,Summer,1900,Fencing,Men,GOLD,Athlete,France,17,28,103,60.714286
3376,Great Britain,Summer,2012,Swimming,Men,BRONZE,GameTeam,People's Republic of China,10,23,102,43.478261
30726,Sweden,Summer,1912,Wrestling,Men,SILVER,Athlete,Finland,7,8,26,87.500000


In [ ]:
# Q2 Data vis

# which country has hosted the games the most? 

In [ ]:
# Q2 Data vis

# reshow the most ever won medals, lets compare how the 2 plots differ 

In [ ]:
# Q3: Which sport is most female and male dominated? 

In [ ]:
# Q3 EDA

# create a bar graph with average participants grouped by sex for each sport 
# will first need to determine the unique sports 